In [ ]:
%%HTML
<style>
.container { width:100% }
</style>

# ETL

## Import der benötigten Bibliotheken

In [ ]:
import pprint
import nltk
import csv
import operator
import re
import pandas    as pd
from pymongo     import MongoClient
from nltk        import word_tokenize
from nltk.corpus import stopwords
from string      import ascii_letters
from pathlib     import Path

## Laden der Sätze und falls notwendig Konvertierung der .txt-Datenquellen in .csv-Dateien

In [ ]:
if Path('sentences_de.csv').is_file():
    sentences_csv = pd.read_csv ('sentences_de.csv', names=['id', 'sentence'], header = 0)
elif Path('deu_news_2015_3M-sentences.txt').is_file():
    sentences_csv = pd.read_csv ('deu_news_2015_3M-sentences.txt', names=['id', 'sentence'], header = 0, delimiter='\t')
    sentences_csv.to_csv ('sentences_de.csv', index=None)
else:
    print("Die benötigten Daten existieren nicht. Bitte lade sie von folgender Seite runter: https://www.kaggle.com/rtatman/3-million-german-sentences")

## Laden der Stopwords

In [ ]:
nltk.download('stopwords')
stop = set(stopwords.words('german'))

## Entfernen der Stopwords und allen Zeichen, die keine Ascii-Buchstaben sind

In [ ]:
def remove_stopwords_and_nonascii(sentences):
    queue = []
    for sentence in sentences:
        just_ascii = re.sub(r"[^{}]".format(ascii_letters + 'äöüÄÖÜß'), ' ', sentence) 
        result = ' '.join([i for i in just_ascii.lower().split() if i not in stop and i != ' '])
        if result != ' ': queue.append(result)
    return queue

In [ ]:
nostop = remove_stopwords_and_nonascii(sentences_csv['sentence'])
nostop = pd.DataFrame(nostop, columns=['sentence'])
nostop.to_csv('sentences_preprocessed.csv', index=False)

## Verbindung mit mongoDB

In [ ]:
connection_string      = "mongodb://localhost:27017"
db                     = MongoClient(connection_string)
sentences              = db.dw.sentences
sentences_preprocessed = db.dw.sentences_preprocessed
bigrams                = db.dw.bigrams

## Sentences und Stopwords der Database in mongoDB hinzufügen

`save_to_mongo()` speichert Listen von Dictionarys oder DataFrames in einer Collection. Dafür kann zunächst die Collection gelöscht werden (`drop_collection = True`). Sonst werden die Daten zusätzlich in die Collection eingefügt.

In [ ]:
def save_to_mongo(collection, data, drop_collection):
    if drop_collection:
        collection.drop()
    if isinstance(data, pd.DataFrame): data = data.to_dict('records')
    result = collection.insert_many(data)
    print('%d rows are saved to "%s" collection successfully!' % (len(result.inserted_ids), collection.name))

In [ ]:
save_to_mongo(sentences, sentences_csv, True)

In [ ]:
save_to_mongo(sentences_preprocessed, nostop, True)

# Analyse

## Extraktion der Daten
`get_column_of_collection()` liest alle Datensätze aus einer Spalte einer Collection und gibt eine Liste mit diesen Datensätzen zurück.

In [ ]:
def get_column_of_collection(collection, column):
    documents = []
    cursor = collection.find({})
    for document in cursor:
        documents.append(document[column])
    return documents

In [ ]:
extracted_documents = get_column_of_collection(sentences_preprocessed, 'sentence')

## Finden der häufisten Bigrams (= 2-Wort-Kombinationen)

`count_bigrams()` nimmt eine Liste von Sätzen, bildet alle Bigrams, und speichert diese mit ihrer Häufigkeit in einem Dictionary.

In [ ]:
def count_bigrams(data):
    bigram_dict = dict()
    for i in range(len(data)):
        text = data[i]
        bigrams = list(nltk.bigrams(text.split()))
        for bigram in bigrams:
            if bigram in bigram_dict:
                bigram_dict[bigram] += 1
            else:
                bigram_dict[bigram] = 1
    return bigram_dict

`get_k_most_frequent()` nimmt ein zuvor mit `count_bigrams()`erstelltes Dictionary, sortiert es nach den Haüfigkeiten, und gibt die k Elemente mit der größten Häufigkeit zurück.

In [ ]:
def get_k_most_frequent(dictionary, k):
    sorted_dictionary =  dict(sorted(dictionary.items(), key=operator.itemgetter(1),reverse=True))
    result = sorted_dictionary.items()
    return list(result)[:k]

`pretty_print_most_frequent()` nimmt die von `get_k_most_frequent()` ausgegebenen Datensätze und gibt sie schön formatiert aus.

In [ ]:
def pretty_print_most_frequent(data):
    for i in range(len(data)):
        words, frequency = data[i]
        print('Platz ', i+1, ': Die Wortkombination "', ' '.join(words), \
              '" kommt in den Daten ', frequency, ' mal vor.', sep='')

`bigrams_to_dict()` bereitet die Bigrams so vor, dass sie in einer Collection gespeichert werden können.

In [ ]:
def bigrams_to_dict(bigrams):
    result = []
    for item in bigrams:
        dictionary = {'bigram': item, 'frequency': bigrams[item]}
        result.append(dictionary)
    return result

`get_k_most_frequent_bigrams()` ruft die zuvor implementierten Funktionen auf. Zunächst werden die Bigrams gebildet und gezählt. Diese werden dann formatiert und in der Datenbank gespeichert. Anschließend werden die k häufigsten Bigrams schön ausgedruckt und schließlich auch zurückgegeben.

In [ ]:
def get_k_most_frequent_bigrams(data, k, collection_to_save):
    bigram_dict   = count_bigrams(data)
    saveable_dict = bigrams_to_dict(bigram_dict)
    save_to_mongo(collection_to_save, saveable_dict, True)
    most_frequent = get_k_most_frequent(bigram_dict, k)
    pretty_print_most_frequent(most_frequent)
    return most_frequent

In [ ]:
%%time
most_frequent = get_k_most_frequent_bigrams(extracted_documents, 10, bigrams)

# ToDos
- Einlesen von csv ohne Konvertierung ✓
- Einfügen von sentences_preprocessed in DB ohne Umweg über DataFrame und .csv ✓
- Spaltenname ohne Umbenennung ✓
- Daten für Analyse aus DB ziehen ✓
- Das Ergebnis (komplettes Bigram-Dict?) in der DB speichern ✓
- Weitere Analysen?
- Satzzeichen bei Stopwörtern hizufügen ✓